# SRE Scrape
<strong>Notebook Objective:</strong> Scrape sre.google books and create input.txt file

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
base_url = "https://sre.google"

In [3]:
input_path = "./data/sre.google/"

def scrape_page(url):
    req = requests.get(f"{base_url}/{url}")
    soup = BeautifulSoup(req.text, "html.parser")
    text_list = [val.text for val in soup.find("div", {"id": "maia-main"}).find_all("p")]
    next_url = soup.find("div", {"class": "next"}).find("a", href=True)["href"]
    return text_list, next_url

In [4]:
def scrape_book(start_url, end_url):
    
    n = 0
    next_url = start_url
    
    text_list = []
    while next_url != end_url:

        text_list_tmp, next_url = scrape_page(next_url)
        text_list += text_list_tmp
        
    return text_list

In [5]:
workbook_text_list = scrape_book(
    "/workbook/foreword-I/",
    "/workbook/index/"
)

In [6]:
len(workbook_text_list)

2852

In [7]:
srebook_text_list = scrape_book(
    "/sre-book/foreword/",
    "/sre-book/bibliography/"
)

In [8]:
len(srebook_text_list)

2646

In [9]:
complete_text_list = workbook_text_list + srebook_text_list

In [10]:
list_of_list = [
    (
        val.replace("\x80", "").replace("\x99", "").replace("â", "'").replace("\'\x9c", '"').replace("\'\x9d", '"').replace("\'\x94", "—").strip()
    ).split("\n") for val in complete_text_list
]

In [11]:
list_of_list[:2]

[['Mark Burgess'],
 ["Having introduced the first SRE book for O'Reilly, I am honored to be invited back for the sequel. In this book, the writing team is leaving the history of the first book to speak for itself and reaching out to a broader audience, offering direct experiences, case studies, and informal guidance. The broad themes will be familiar to anyone in IT, perhaps relabeled and reprioritized, and with a modern sense of business awareness. In place of technical descriptions, here we have user-facing services and their promises or objectives. We see human-computer systems originate from within the evolving business, intrinsic to its purpose, rather than as foreign meteorites impacting an unsuspecting and pristine infrastructure. Cooperation of all human-computer parts is the focus. Indeed, the book might be summarized as follows:"]]

In [12]:
text_list = [
    item for sublist in list_of_list for item in sublist if len(item) > 100
]

In [13]:
text_list[:2]

["Having introduced the first SRE book for O'Reilly, I am honored to be invited back for the sequel. In this book, the writing team is leaving the history of the first book to speak for itself and reaching out to a broader audience, offering direct experiences, case studies, and informal guidance. The broad themes will be familiar to anyone in IT, perhaps relabeled and reprioritized, and with a modern sense of business awareness. In place of technical descriptions, here we have user-facing services and their promises or objectives. We see human-computer systems originate from within the evolving business, intrinsic to its purpose, rather than as foreign meteorites impacting an unsuspecting and pristine infrastructure. Cooperation of all human-computer parts is the focus. Indeed, the book might be summarized as follows:",
 "Keeping promises reliably (to all stakeholders) depends on the stability of all their dependencies, of intent, and of the lives of the people involved (e.g., see Thi

In [56]:
len(text_list)

4133

In [57]:
with open("../data/sre.google_char/input.txt", "w") as txt_file:
    txt_file.write("\n".join(text_list))